In [ ]:
from google.colab import drive  # Se importa el drive donde estan todos los .py y las imagenes utilizadas, esta celda debe ser modificada en caso de que se quiera probar el codigo en otro equipo.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/sensxciones/Proyecto_Humedales.git

Cloning into 'Proyecto_Humedales'...
remote: Enumerating objects: 1407, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 1407 (delta 28), reused 72 (delta 24), pack-reused 1327
Receiving objects: 100% (1407/1407), 106.02 MiB | 38.98 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [ ]:
! cd Proyecto_Humedales && git pull

Already up to date.


In [ ]:
# Se importar todos los modulos y funciones necesarios
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import sys
sys.path.append('Proyecto_Humedales')

#La siguiente importacion es de un codigo diseñado para el proyecto patimetria:
from duckies_dataset import DuckieDataset, Rescale
#Los siguientes modulos son versiones modificadas de .py de terceros adapatados para utilizar en el proyecto patimetria.
# Los modulos originales se encuentran en el siguiente git: https://github.com/adambielski/siamese-triplet
from networks import VGGEmbeddingNet, EmbeddingNet, TripletNet
from trainer import fit, train_epoch, test_epoch
from datasets import TripletMNIST
from losses import TripletLoss

In [ ]:
# Se cargan 2 modelos a utilizar.
vgg_model = VGGEmbeddingNet()
# emb_net = EmbeddingNet()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 144MB/s]


In [ ]:
# Se define una transformacion que convierte las imagenes a tensores de tamaño [224, 224]
trans = transforms.Compose([Rescale(224),
                            transforms.ToTensor()])

In [ ]:
# Se definen los datasets de entrenamiento y de testeo.
train_dataset = DuckieDataset("/content/drive/MyDrive/bird-dataset/train",train = False, transform= trans)
test_dataset = DuckieDataset("/content/drive/MyDrive/bird-dataset/test",train = False, transform= trans)
valid_dataset = DuckieDataset("/content/drive/MyDrive/bird-dataset/valid",train = False, transform= trans)


FileNotFoundError: ignored

In [ ]:
cuda = torch.cuda.is_available()
%matplotlib inline
# Set up data loaders

triplet_train_dataset = TripletMNIST(train_dataset) # Returns triplets of images
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 32
kwargs = {'num_workers': 2, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters

margin = 1.
# embedding_net = EmbeddingNet()
model = TripletNet(vgg_model)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 10
log_interval = 500

In [ ]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval) #Se empieza a entrenar la red.

In [ ]:
def resume(model, filename):
    if cuda:
        model.load_state_dict(torch.load(filename))
    else:
        model.load_state_dict(torch.load(filename, map_location=torch.device('cpu')))

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/bird-dataset/epoch-10.pth')

In [ ]:
model_cargado = TripletNet(vgg_model)
if cuda:
    model_cargado.cuda()

resume(model_cargado, '/content/drive/MyDrive/bird-dataset/epoch-10.pth')

In [ ]:
# Define functions to extract embeddings using diferent datasets
def extract_embeddings(dataset, model, dims = 1024, opt = None):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataset), dims))
        labels = list()
        for k, it_ in enumerate(dataset):
            images = it_[0].unsqueeze_(0)
            images = images.cuda()
            target = it_[1]
            if opt is None:
                aux = model.get_embedding(images).data.cpu().numpy()
            else:
                aux = model.get_embedding(images, opt).data.cpu().numpy()
            embeddings[k] = aux.reshape(1, dims)
            labels.append(target)
    return embeddings, labels


In [ ]:
embeddings, labels = extract_embeddings(train_dataset, vgg_model, dims = 512)

In [ ]:
len(test_dataset.data)

In [ ]:
print(labels)

In [ ]:
# n_closest_images: (str), (int) ---> list
# Funcion que recibe la ruta de una imagen y de manera opcional un numero entero"n" (por defecto 5), que escribe 3 listas y entrega 1:
# Near_list: lista que contiene las n distancias mas cortas con respecto a la imagen entregada
# List_labels: lista que contiene las etiquetas correspondientes a las "n" imagenes con distancias mas cortas. Esta lista es entregada por la funcion.
# Label_ubication_list: lista que entrega la posicion de las "n" imagenes mas cercanas con respecto a su etiqueta.

def n_closest_images(image, n=5):
  img = Image.open(image).convert('RGB')
  img = trans(img)
  img = img.cuda()
  emb_img = vgg_model(img.unsqueeze(0))
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(n):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  #print (neat_list[:n], list_labels, label_ubication_list)
  return list_labels

In [ ]:
n_closest_images("/content/drive/MyDrive/bird-dataset/test/ABBOTTS BABBLER/1.jpg")[0]

In [ ]:
# closest_image_after_rotation: (str) ---> list
# Funcion que recibe la ruta de una imagen, escribe 3 numeros y entrega 1:
# neat_list[:1]: Corresponde a la distancia mas corta entre la imagen entregada y una del dataset
# list_labels: Corresponde a la lista con las etiquetas de las imagenes del dataset, ordenadas desde las imagenes mas cercanas hasta las mas alejadas con respecto a la imagen entregada.
# esta lista es entregada por la funcion.
# Label_ubication_list: lista que entrega la posicion de las imagenes con respecto a su etiqueta. (Nota: si en la etiqueta tiene "n" imagenes, 0 representa la primera imagen y
# "k" representa la "k+1"-esima imagen)"
def closest_image_after_rotation(image):
  img = Image.open(image).convert('RGB')
  flip = transforms.RandomHorizontalFlip(p=1)
  img = flip(img)
  img = trans(img)
  img = img.cuda()
  emb_img = vgg_model(img.unsqueeze(0))
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(1):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  return list_labels

In [ ]:
# percentage_check (dataset) ---> ()
# Funcion que recibe un dataset y rota las imagenes del mismo para luego pasarlas por la red y verificar cuantas imagenes son clasificadas correctamente en su etiqueta.
# El resultado es escrito como un porcentaje
def percentage_check(dataset):
  Right = 0
  for i in range(len(dataset.data)):
    img = dataset.data[i]
    label= closest_image_after_rotation(img)
    if label[0] == dataset.targets[i]:
      Right += 1
  Total = len(dataset.targets)
  Percentage = 100.0*Right/Total
  print ("% " + str(Percentage) +" de las imagenes fueron clasificadas correctamente.")



In [ ]:
def percentage_check_from_valid(dataset):
  Right1 = 0
  Right3 = 0
  Right5 = 0
  for i in range(len(dataset.data)):
    img = dataset.data[i]
    label= n_closest_images(img, n=10)
    if label[0] == dataset.targets[i]:
      Right1 += 1
    if dataset.targets[i] in label[:3]:
      Right3 += 1
    if dataset.targets[i] in label[:5]:
      Right5 += 1
  Total = len(dataset.targets)
  Percentage1 = 100.0*Right1/Total
  Percentage3 = 100.0*Right3/Total
  Percentage5 = 100.0*Right5/Total
  print ("% " + str(Percentage1) +" de las imagenes fueron clasificadas correctamente en top 1.")
  print ("% " + str(Percentage3) +" de las imagenes fueron clasificadas correctamente en top 3.")
  print ("% " + str(Percentage5) +" de las imagenes fueron clasificadas correctamente en top 5.")

In [ ]:
percentage_check_from_valid(valid_dataset)

In [ ]:
percentage_check(test_dataset)

In [ ]:
test_dataset.class_to_idx